In [ ]:
for year in range(START_YEAR, END_YEAR + 1):
    year_probs = ee.Image()
    for threshold_idx in range(25):
        threshold = (threshold_idx + 1) * 5
        current_prob_list = []
        future_prob_list = []
        successes = ee.Image.constant(0)
        failures = ee.Image.constant(0)
        year_threshold_images_list = ee.List([])

        alphabeta = priors[threshold_idx]
        alpha = parameter_prime(alphabeta.select('alpha'), year, threshold, False)
        beta = parameter_prime(alphabeta.select('beta'), year, threshold, True)

        for model in MODELS:
            tasmax_model = tasmax_rcp85.filterMetadata('model', 'equals', model)
            ehe_year_model = ehe(tasmax_model.filterDate(str(year) + '-01-01', str(year) + '-12-31'), model)
            successes = successes.add(ehe_year_model.gte(threshold).multiply(ehe_year_model.lt(threshold + 5)))
            failures = failures.add(ehe_year_model.lt(threshold).add(ehe_year_model.gte(threshold + 5)).min(1))
            year_threshold_images = (successes.add(alpha).subtract(1)).divide(successes.add(failures).add(beta).add(alpha).subtract(2))
            year_threshold_images_list = year_threshold_images_list.add(year_threshold_images)
        year_threshold_meanprob = ee.ImageCollection(year_threshold_images_list).mean().rename('threshold_' + str(threshold))
        year_probs = year_probs.addBands(year_threshold_meanprob)
    year_probs = year_probs.set('year', year)
    year_probs = year_probs.select(
        year_probs.bandNames().filter(
            ee.Filter.stringStartsWith('item', 'threshold_')
        )
    )
  
    task = ee.batch.Export.image.toAsset(**{
        'image': year_probs,
        'description': 'noncumul_ehe_MAPprob_prime01_' + str(year),
        'assetId': 'users/tedwongwri/MAP_probs/noncumul_ehe_MAPprob_prime01_' + str(year),
        'region': wholeGlobe,
        'crs': 'EPSG:4326',
        'dimensions': '1440x720'
    })
    task.start()